<a href="https://colab.research.google.com/github/Bimal022/khel_darpan/blob/master/AtheletesMedalPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import tensorflow as tf

In [2]:
df = pd.read_csv('/content/athletes.csv')
df.head(5)

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


In [3]:
del df['dob'], df['id'], df['name']

In [5]:
df.head(5)

,nationality,sex,height,weight,sport,gold,silver,bronze
0,ESP,male,1.72,64.0,athletics,0,0,0
1,KOR,female,1.68,56.0,fencing,0,0,0
2,CAN,male,1.98,79.0,athletics,0,0,1
3,MDA,male,1.83,80.0,taekwondo,0,0,0
4,NZL,male,1.81,71.0,cycling,0,0,0


In [12]:
df.isnull().sum()

nationality      0
sex              0
height         330
weight         659
sport            0
gold             0
silver           0
bronze           0
dtype: int64

In [13]:
df['nationality'] = df['nationality'].astype(str)

In [20]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(df.iloc[:,2:4])
df.iloc[:,2:4] = imputer.transform(df.iloc[:,2:4])

In [21]:
df.isnull().sum()

nationality    0
sex            0
height         0
weight         0
sport          0
gold           0
silver         0
bronze         0
dtype: int64

In [22]:
X = df.iloc[:, 0:5].values
Y = df.iloc[:,[5,6,7]].values

In [23]:
X = X.transpose()

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[0]= le.fit_transform(X[0])

In [25]:
X

array([[59, 102, 33, ..., 38, 202, 158],
       ['male', 'female', 'male', ..., 'male', 'male', 'male'],
       [1.72, 1.68, 1.98, ..., 1.85, 1.6, 1.85],
       [64.0, 56.0, 79.0, ..., 100.0, 56.0, 70.0],
       ['athletics', 'fencing', 'athletics', ..., 'wrestling',
        'weightlifting', 'athletics']], dtype=object)

In [26]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[4]= le.fit_transform(X[4])

In [27]:
X

array([[59, 102, 33, ..., 38, 202, 158],
       ['male', 'female', 'male', ..., 'male', 'male', 'male'],
       [1.72, 1.68, 1.98, ..., 1.85, 1.6, 1.85],
       [64.0, 56.0, 79.0, ..., 100.0, 56.0, 70.0],
       [2, 9, 2, ..., 27, 26, 2]], dtype=object)

In [28]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[1] = le.fit_transform(X[1])

In [29]:
X

array([[59, 102, 33, ..., 38, 202, 158],
       [1, 0, 1, ..., 1, 1, 1],
       [1.72, 1.68, 1.98, ..., 1.85, 1.6, 1.85],
       [64.0, 56.0, 79.0, ..., 100.0, 56.0, 70.0],
       [2, 9, 2, ..., 27, 26, 2]], dtype=object)

In [30]:
gender = df['sex'].values
nationality = df['nationality'].values
sports = df['sport'].values

In [31]:
cat_gen = dict(zip(gender ,X[1]))
cat_nat = dict(zip(nationality, X[0]))
cat_spt = dict(zip(sports, X[4]))

In [32]:
cat_data = [cat_nat, cat_gen, cat_spt]

In [33]:
X = X.transpose()

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

In [35]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, Y_train)

LinearRegression()

In [36]:
y_pred = linreg.predict(X_test)

In [37]:
from sklearn.metrics import r2_score
r2_score(Y_test, y_pred)

0.0026661009023385787

In [44]:
def PredictMedals(country, gender, height, weight,sport):
  c = cat_data[0].get(country.upper())
  g = cat_data[1].get(gender.lower())
  s = cat_data[2].get(sport.lower())
  h = height
  w = weight
  list_data = [c,g,h,w,s]
  pred = linreg.predict([list_data])
  # Gold
  if pred[0][0] < 0.036:
    gold = 0
  elif pred[0][0] > 0.036 and pred[0][0] < 0.12:
    gold = 1
  elif pred[0][0] > 0.12 and pred[0][0] < 0.9:
    gold = 2
  elif pred[0][0] > 0.9:
    gold = '2+'
  # Silver
  if pred[0][1] < 0.3:
    silver = 0
  elif pred[0][1] > 0.3 and pred[0][1] < 0.5:
    silver = 1
  elif pred[0][1] > 0.5:
    silver = '1+'
  # Bronze
  if pred[0][2] < 0.21:
    bronze = 0
  elif pred[0][2] > 0.21 and pred[0][2] < 4.5:
    bronze = 1
  elif pred[0][2] > 4.5:
    bronze = '1+'

  prediction = [gold, silver, bronze]
  return prediction

In [46]:
country = 'CAN'
gender = 'male'
height = 1.98
weight = 64
sports = 'athletics'
PredictMedals(country, gender,height, weight, sports)

[1, 0, 0]